In [8]:
pd.set_option('display.max_columns', None)

In [1]:
import pandas as pd

In [3]:
from api import static

In [4]:
live = static()
elements_df = live.get_elements_summary()

In [10]:
elements_df.columns

Index(['chance_of_playing_next_round', 'chance_of_playing_this_round', 'code',
       'cost_change_event', 'cost_change_event_fall', 'cost_change_start',
       'cost_change_start_fall', 'dreamteam_count', 'element_type', 'ep_next',
       'ep_this', 'event_points', 'first_name', 'form', 'id', 'in_dreamteam',
       'news', 'news_added', 'now_cost', 'photo', 'points_per_game',
       'second_name', 'selected_by_percent', 'special', 'squad_number',
       'status', 'team', 'team_code', 'total_points', 'transfers_in',
       'transfers_in_event', 'transfers_out', 'transfers_out_event',
       'value_form', 'value_season', 'web_name', 'minutes', 'goals_scored',
       'assists', 'clean_sheets', 'goals_conceded', 'own_goals',
       'penalties_saved', 'penalties_missed', 'yellow_cards', 'red_cards',
       'saves', 'bonus', 'bps', 'influence', 'creativity', 'threat',
       'ict_index', 'starts', 'expected_goals', 'expected_assists',
       'expected_goal_involvements', 'expected_goals_con

In [13]:
cost_columns = ['code', 'web_name','cost_change_event', 'cost_change_event_fall', 'cost_change_start',
                'cost_change_start_fall', 'now_cost']

In [15]:
cost_changes_now = elements_df.loc[elements_df.cost_change_event != 0, cost_columns]

In [16]:
cost_diff = elements_df[cost_columns]